In [1]:
from datetime import datetime
now1 = datetime.now()
print("Début du script =", now1)

import warnings
warnings.filterwarnings('ignore')

Début du script = 2020-07-28 08:55:39.135524


In [2]:
import os
import sys
import datetime as dt
import pandas as pd
import numpy as np
import statistics as sts

#A executer à la fin, après avoir sauvegarder pour passer en *.py
#!jupyter nbconvert --to script parent_enfant.ipynb

In [3]:
#Connexion base Cassandra
import cassandra
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

auth_provider = PlainTextAuthProvider(username='cassandra', password='cassandra')
cluster = Cluster(['srv-cassandra'], auth_provider=auth_provider)

session = cluster.connect()

insertinto = session.prepare("""
                             INSERT INTO ods.genealogie (id_produit, id_fardeau, date_depart_client, genealogie, id_rouleau, machine_prod, niveau, num_bl, num_cmd_sap, type_produit) 
                             VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
                             """)

# bounded = insertinto.bind(listCass)                
# session.execute(bounded)

In [4]:
rows = session.execute("""SELECT name FROM ods.tags_def WHERE line = 'V3_rv'""")
taglist2 = []
for row in rows :
    taglist2.append(row.name)

In [5]:
# taglist2

In [6]:
debut_bonne_periode = dt.datetime(2020,5,28,15,0)
fin_bonne_periode = dt.datetime(2020,5,28,15,15)

In [7]:
# SELECT name, timestamp, value
nowtemp = datetime.now()

In [8]:
dataz = {'name': [], 'avg': [], 'min': [], 'max': [], 'count': [], 'std1': []}
df_bonne_periode_final = pd.DataFrame(dataz)

for i in range (len(taglist2)):
#     print(i, taglist2[i], str(debut_bonne_periode.year), str(debut_bonne_periode), str(fin_bonne_periode))
    rows_bonne_periode = session.execute("""SELECT name, value
                                                FROM aethertimeseries.datapointsyear 
                                                WHERE name = $$"""+taglist2[i]+"""$$ 
                                                AND year = """+str(debut_bonne_periode.year)+"""

                                                AND timestamp >= $$"""+str(debut_bonne_periode)+"""$$
                                                AND timestamp <= $$"""+str(fin_bonne_periode)+"""$$;
                                                
                                           """)
    
    tag_name = taglist2[i]
    list_value = []
    avg, count, minimum, maximum, std1 = 0, 0, 0, 0, 0
    
    try :
        for row in rows_bonne_periode :
            list_value.append(float(row.value))
        avg = round(sts.mean(list_value),3)
        count = round(len(list_value),3)
        minimum = round(min(list_value),3)
        maximum = round(max(list_value),3)
        std1 = round(np.std(list_value), 3)
        
    except :
        del avg
    
    try :
        list_df = [tag_name, avg, minimum, maximum, count, std1]
        df_bonne_periode_final.loc[len(df_bonne_periode_final)] = list_df
        
    except :
        print(taglist2[i])

In [9]:
now2 = datetime.now()
print("Fin du Script =", now2)

Fin du Script = 2020-07-28 08:55:46.110247


In [10]:
print (now2-nowtemp)

0:00:04.628881


In [11]:
df_bonne_periode_final['minus 1 std'] = df_bonne_periode_final['avg'] - df_bonne_periode_final['std1']
df_bonne_periode_final['minus 2 std'] = df_bonne_periode_final['avg'] - df_bonne_periode_final['std1']*2
df_bonne_periode_final['minus 3 std'] = df_bonne_periode_final['avg'] - df_bonne_periode_final['std1']*3

df_bonne_periode_final['plus 1 std'] = df_bonne_periode_final['avg'] + df_bonne_periode_final['std1']
df_bonne_periode_final['plus 2 std'] = df_bonne_periode_final['avg'] + df_bonne_periode_final['std1']*2
df_bonne_periode_final['plus 3 std'] = df_bonne_periode_final['avg'] + df_bonne_periode_final['std1']*3

In [12]:
df_bonne_periode_final.head()

,name,avg,min,max,count,std1,minus 1 std,minus 2 std,minus 3 std,plus 1 std,plus 2 std,plus 3 std
0,V3.LF.EL.Dancing.roll.1.pv,2.030,2.030,2.030,180.0,0.0,2.030,2.030,2.030,2.030,2.030,2.030
1,V3.LF.EL.Dancing.roll.2.pv,0.074,0.074,0.074,15.0,0.0,0.074,0.074,0.074,0.074,0.074,0.074
2,V3.LF.EL.pression.presseur.P1.EF.pv,0.120,0.120,0.120,15.0,0.0,0.120,0.120,0.120,0.120,0.120,0.120
3,V3.LF.EL.pression.presseur.P1.SF.pv,0.120,0.120,0.120,15.0,0.0,0.120,0.120,0.120,0.120,0.120,0.120
4,V3.LF.EL.pression.presseur.P2.EF.pv,0.110,0.110,0.110,15.0,0.0,0.110,0.110,0.110,0.110,0.110,0.110


In [13]:
df_bonne_periode_final.to_csv("/share-srvcassandra/Divers/bonneperiode/V3/bonneperiode_mai_15h15.csv",  index = False, sep =";", encoding = "latin-1")

In [49]:
path = "/share-srvcassandra/Divers/bonneperiode/"
dirs = os.listdir(path)
for file in dirs:
   print(file)

FT1
T4
V1
V2
V3
